In [1]:
import yfinance as yf

In [5]:
volitile = yf.download("RIVN", start='2022-03-16', end='2022-03-17')
spy=yf.download("GSPC")

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RIVN,RIVN,RIVN,RIVN,RIVN
Date,,,,,
2022-03-16,41.580002,42.099998,37.650002,38.5,23752200
